In [2]:
#this noetbook focus on the evaluation of summary quality using LLM
from datasets import load_dataset
import pandas as pd
import re
from tqdm import tqdm
import numpy as np
import json
# Set OpenAI's API key and API base to use vLLM's API server.
from scipy.stats import kendalltau
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8222/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

llm_model = "Qwen/Qwen1.5-72B-Chat"




In [3]:
#define function block

#genreate prompt
def generate_prompt(few_shot: int, aspect:str, refer_news = None, refer_summary = None, refer_score = None):
    prompt = "You are a helpful summary assistant."
    if(aspect!="Faithfulness"):
        base_prompt = "In this task, you will be provided with a news article and a generated summary.\n"\
                "Your task is to rate the " + aspect +" of the generated summary with a score from 1 to 5, "\
                "where 1 is the lowest and 5 is the highest.\n"\
                "Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.\n"
        if(few_shot==0):
            prompt = base_prompt+ "Example Response:\n"+aspect+" (1-5): 3"
        else:
            shot_num = len(refer_news)
            if shot_num>2:
                shot_num = 2
            if(shot_num==1):
                base_prompt+="This is an example:\n"
            else:
                base_prompt+="These are examples:\n"
            for i in range(shot_num):
                prompt = base_prompt + "News: "+refer_news[i]+"\nGenerated summary: "+refer_summary[i]+"\n"+aspect+" (1-5): "+str(refer_score[i])+"\n"
    else:
        base_prompt = "In this task, you will be provided with a news article and a generated summary.\n"\
                "Your task is to rate the " + aspect +" of the generated summary with a score 0 or 1, "\
                "where 1 is not fact and 1 is fact.\n"\
                "Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.\n"
        
        if(few_shot==0):
            prompt = base_prompt+ "Example Response:\n"+aspect+" (0,1): 1"
        else:
            shot_num = len(refer_news)
            if shot_num>2:
                shot_num = 2
            if(shot_num==1):
                base_prompt+="This is an example:\n"
            else:
                base_prompt+="These are examples:\n"
            for i in range(shot_num):
                prompt = base_prompt + "News: "+refer_news[i]+"\nGenerated summary: "+refer_summary[i]+"\n"+aspect+" (0,1): "+str(refer_score[i])+"\n"
                                   
    return prompt
    

#get socre using openai api
def score_api_chat(client, model_name, prompt, user_input, temperature = 0):
    chat_response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": user_input},
                
            ],
            temperature=temperature
        )
        # "\n\nReference summary: "
        #         +reference_summary+
    res = chat_response.choices[0].message.content
        
    match = re.search(r':\s*(\d+)', res)   
    score = match.group(1) if match is not None else 1
    return int(score)


def loop_score_api_chat(news_list, summary_list, client, model_name, aspect, 
                        refer_news = None, refer_summary = None, refer_score = None,temperature = 0):
    
    score_list = []
    prompt = ""
    if(refer_news is None):
        prompt = generate_prompt(0, aspect)  
    else:
        prompt = generate_prompt(1, aspect, refer_news, refer_summary, refer_score)

    
    for i in range(len(news_list)):
            news = news_list[i]
            summary = summary_list[i]
            user_input = "News: "+news+"\n\Generated summary: "+summary
            score_list.append(score_api_chat(client, model_name, prompt, user_input))
    return score_list   

def correlation_score(dict1, dict2):
    #system level
    tmp_list1 = []
    tmp_list2 = []
    for i in dict1.keys():
        tmp_list1.append(np.mean(dict1[i]))
        tmp_list2.append(np.mean(dict2[i]))
    print("correlation of system level is ", kendalltau(tmp_list1, tmp_list2)[0])
    
    #summary level
    total_corr = 0
    
    for i in dict1.keys():
        total_corr+=kendalltau(dict1[i], dict2[i])[0]
    print("correlation of summary level is ", total_corr/len(dict1.keys()))
    

In [63]:
#load the dataset to processing

cnndm_liker = pd.read_json('/home/xbr/LLM/benchmark_llm_summarization/likert_evaluation_results_cnndm_average.json')

xsum_liker = pd.read_json('/home/xbr/LLM/benchmark_llm_summarization/likert_evaluation_results_xsum_average.json')

summeval = pd.read_json('./filter_annotations_.jsonl')


target_dataset = summeval
template_dataset = summeval
template_index = "None"
aspect = "relevance"

tag_aspect = "expert_relevance"


model_list = list(set(target_dataset['model'].tolist()))

model_list.remove('M0')

model_eva_dict= {}
human_eva_dict = {}
# total_summary_level = 0

# ['Relevance', 'Coherence', 'Consistency', 'Faithfulness', 'Informativeness']



# template_news = [template_dataset[(template_dataset['model']=="reference" )].to_dict(orient='records')[i]['article'] for i in range(2,4)]
# template_summary = [template_dataset[(template_dataset['model']=="reference" )].to_dict(orient='records')[i]['summary'] for i in range(2,4)]
# template_score = [template_dataset[(template_dataset['model']=="reference" )].to_dict(orient='records')[i][aspect] for i in range(2,4)]

template_news = [template_dataset[(template_dataset['model']=="M0" )].to_dict(orient='records')[i]['article'] for i in range(1)]
template_summary = [template_dataset[(template_dataset['model']=="M0" )].to_dict(orient='records')[i]['summary'] for i in range(1)]
template_score = [template_dataset[(template_dataset['model']=="M0" )].to_dict(orient='records')[i][tag_aspect] for i in range(1)]

# template_news = None
# template_summary = None
# template_score = None



for m in model_list:
    print("evaluating model: ", m)
    tmp_dataset = target_dataset[(target_dataset['model']==m )]
    
    tmp_news_list = tmp_dataset['article'].tolist()
    
    tmp_summary_list = tmp_dataset['summary'].tolist()
    tmp_score_list = tmp_dataset[tag_aspect].tolist()
    score_list = loop_score_api_chat(tmp_news_list, tmp_summary_list, client, llm_model, aspect, 
                                     refer_news = template_news, refer_summary = template_summary, refer_score = template_score)
    
    model_eva_dict[m] = score_list
    human_eva_dict[m] = tmp_score_list
    

    
#save the result
with open('./evaluate_result/'+'qwen72_summeval_'+tag_aspect+'_template_summeval_'+str(template_index)+'_eva.json', 'w') as fp:
    json.dump(model_eva_dict, fp)
    
with open('./evaluate_result/'+'summeval_human_eva_dict.json', 'w') as fp:
    json.dump(human_eva_dict, fp)

  

evaluating model:  M20
evaluating model:  M5
evaluating model:  M15
evaluating model:  M8
evaluating model:  M23
evaluating model:  M11
evaluating model:  M10
evaluating model:  M17
evaluating model:  M14
evaluating model:  M9
evaluating model:  M2
evaluating model:  M13
evaluating model:  M22
evaluating model:  M12
evaluating model:  M1


In [64]:
correlation_score(model_eva_dict, human_eva_dict)
# for i in model_eva_dict.keys():
#     print(i, ":", np.mean(model_eva_dict[i]), np.mean(human_eva_dict[i]))        

#cnndm
#index 0
# correlation of system level is  0.813119124543455
# correlation of summary level is  0.1472597601736428

#idnex 1
# correlation of system level is  0.7777777777777779
# correlation of summary level is  0.20898023319686118

#index 2
# correlation of system level is  0.33333333333333337
# correlation of summary level is  0.08772663469686875

#index 4
# correlation of system level is  0.6954252406248034
# correlation of summary level is  nan

#index None
# correlation of system level is  0.724262721160196
# correlation of summary level is  nan   

#index 0&1
# correlation of system level is  0.6997042637035166
# correlation of summary level is  0.24644284212607812



#xsum
#None
# correlation of system level is  0.7852067275033966
# correlation of summary level is  0.24612884627714107

#index 1
# correlation of system level is  0.7158720324782436
# correlation of summary level is  0.2570950849356606

#index 2
# correlation of system level is  0.2370435403783839
# correlation of summary level is  0.017176762945798965

#index 1 2
# correlation of system level is  0.7306322805705787
# correlation of summary level is  0.28464519005709527

#index 3
# correlation of system level is  0.7896732729399182
# correlation of summary level is  0.2844474928173168

# #index 2 3
# correlation of system level is  0.8044335210322533
# correlation of summary level is  0.2894187475986224



#summeval
#None
# correlation of system level is  0.8140776760988245
# correlation of summary level is  nan
#index 1
# correlation of system level is  0.6315861769395361
# correlation of summary level is  0.32975744251357864

correlation of system level is  0.6315861769395361
correlation of summary level is  0.32975744251357864


In [8]:
# dict1 = json.load(open('./evaluate_result/qwen72_cnndm_relevance_template_cnndm_2_eva.json'))

# dict2 = json.load(open('./evaluate_result/qwen72_cnndm_relevance_template_cnndm_0_1_eva.json'))

# correlation_score(dict1, dict2)


correlation of system level is  0.21123147583502386
correlation of summary level is  0.16372567961293896


In [ ]:
#尝试最高分挑一个，最低分挑一个，然后看能不能改善结果

